In [202]:
%load_ext autoreload
%autoreload 2

from shrimpgrad.tensor import Tensor
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [209]:
x = Tensor.rand(2,2)

In [210]:
x = Tensor.randn(3,3)
x

tensor([[1.1483342096275484,
  1.2021448757683826,
  -0.5122828778128984],
 [0.06477125864115552,
  0.2229293288125177,
  -1.1808725813725525],
 [1.7327131114862218,
  -0.6675351892976191,
  1.392259489701388]])

In [211]:
import numpy as np

In [213]:
a = np.arange(3*4*5*6).reshape((3,4,5,6))
b = np.arange(3*4*5*6)[::-1].reshape((5,4,6,3))

In [215]:
x = a.dot(b)
x.shape

(3, 4, 5, 5, 4, 3)

In [217]:
a = np.ones([9, 5, 7, 4])
c = np.ones([9, 5, 4, 3])
x = a.dot(c)

In [218]:
x.shape

(9, 5, 7, 9, 5, 3)

In [241]:
self = np.random.rand(2, 3, 4)  # Shape: (2, 3, 4)
w = np.random.rand(5, 4, 4, 2)  # Shape: (5, 4, 3, 2)

In [242]:
n1, n2 = len(self.shape), len(w.shape)
assert n1 != 0 and n2 != 0, f"both arguments to matmul need to be at least 1D, but they are {n1}D and {n2}D"
# assert (L:=self.shape[-1]) == (R:=w.shape[-min(n2, 2)]), f"Input Tensor shapes {self.shape} and {w.shape} cannot be multiplied ({L} != {R})"
x = self.reshape(*self.shape[0:-1], *[1]*min(n1-1, n2-1, 1), self.shape[-1])
w = w.reshape(*w.shape[0:-2], *[1]*min(n1-1, n2-1, 1), *w.shape[-min(n2, 2):])
w.shape

(5, 4, 1, 4, 2)

In [246]:
w = w.transpose(0,1,2,4,3)

In [247]:
x.shape, w.shape

((2, 3, 1, 4), (5, 4, 1, 2, 4))

In [248]:
x*w

ValueError: operands could not be broadcast together with shapes (2,3,1,4) (5,4,1,2,4) 

In [250]:
from tinygrad import Tensor

In [291]:
x = Tensor([1,0,0,1]).reshape(2,2)
y = Tensor([4,1,2,2]).reshape(2,2)
x._broadcast_to((2,2,2))

AttributeError: 'Tensor' object has no attribute '_broadcast_to'

In [259]:
a.shape, w.shape
(1,1,1,4)
(5,4,2,4)

((4,), (5, 4, 4, 2))

In [265]:
i =a*w
i.shape

(5, 4, 2, 4)

In [264]:
i.sum(-1)

<Tensor <LB METAL (5, 4, 2) contig:True ShapeTracker(views=(View(shape=(5, 4, 2), strides=(8, 2, 1), offset=0, mask=None, contiguous=True),))> on METAL with grad None>

In [270]:
x = np.arange(4).reshape(2,2)

y = x.sum(axis=0, keepdims=True)
y.shape # (1,2)
x = np.arange(4).reshape(2,2)
y = x.sum(axis=1, keepdims=False)
y, y.shape

(array([1, 5]), (2,))

In [278]:
x = np.arange(2*3*4).reshape(2,3,4)
y = x.sum(axis=2)

In [279]:
x

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [280]:
y

array([[ 6, 22, 38],
       [54, 70, 86]])

In [286]:
import operator
from functools import reduce

reduce(operator.add, (1,2,3,4))

10

In [296]:
x = np.array([1,0,0,1]).reshape(2,1,2)
y = np.array([4,2,2,2]).reshape(1,2,2)

In [306]:
x = np.broadcast_to(x, (2,2,2))


In [307]:
y = np.broadcast_to(y, (2,2,2))

In [309]:
x.shape, y.shape, x.strides, y.strides, x.dtype

((2, 2, 2), (2, 2, 2), (16, 0, 8), (0, 16, 8), dtype('int64'))

In [300]:
z.strides

AttributeError: 'numpy.broadcast' object has no attribute 'strides'